In [1]:
# załadowanie bibliotek

import pandas as pd
import numpy as np

In [2]:
# wczytanie danych

dane = pd.ExcelFile('data_to_agg.xlsx').parse('aggregation_example')
dane

,imię,płeć,zawód,wiek,staż
0,Marcin,M,elektryk,36,23
1,Ewa,K,programista,33,2
2,Ola,K,programista,25,12
3,Tomek,M,sprzedawca,24,1
4,Ania,K,elektryk,29,29
5,Elwira,K,sprzedawca,55,48
6,Grzegorz,M,elektryk,33,32
7,Jan,M,sprzedawca,27,9
8,Filip,M,elektryk,51,23
9,Róża,K,programista,49,52


# Funkcje agregujące - Statystyki opisowe

### Miary położenia

**Miary tendencji centralnej:**

In [3]:
print(dane['wiek'].mean())                                 # średnia
print(dane['wiek'].median())                               # mediana
print(dane['wiek'].mode())                                 # dominanta

36.2
33.0
0    33
Name: wiek, dtype: int64


**Inne miary położenia:**

In [4]:
print(dane['wiek'].quantile(0.25))                          # kwantyl rzędu 0.25 (1. kwartyl)
print(dane['wiek'].quantile([0, 0.25, 0.5, 0.75, 1]))       # minimum, 1. kwartyl, 2.kwartyl, 3. kwartyl, maksimum

27.5
0.00    24.00
0.25    27.50
0.50    33.00
0.75    45.75
1.00    55.00
Name: wiek, dtype: float64


In [5]:
print(dane['wiek'].min())                                   # minimum  - kwantyl rzędu 0
print(dane['wiek'].median())                                # mediana  - kwantyl rzędu 0.5
print(dane['wiek'].max())                                   # maksimum - kwantyl rzędu 1

24
33.0
55


### Miary rozproszenia

**Miary rozproszenia wokół miary tendencji centralnej:**

In [ ]:
print(dane['wiek'].var())                                   # wariancja nieobciążona
print(dane['wiek'].std())                                   # odchylenie standardowe

### Miary koncentracji

In [ ]:
print(dane['wiek'].kurt())                                  # kurtoza

### Miary asymetrii

In [ ]:
print(dane['wiek'].skew())                                  # skośność 

---

Podsumowanie powyższych miar daje funkcja **describe**:

In [6]:
dane['wiek'].describe()                                     # najważniejsze statystyki opisowe

count    10.000000
mean     36.200000
std      11.390054
min      24.000000
25%      27.500000
50%      33.000000
75%      45.750000
max      55.000000
Name: wiek, dtype: float64

Można też samodzielnie wybrać kilka miar i skorzystać z funkcji **aggregate**:

In [8]:
dane['wiek'].aggregate(['min', 'max', 'count'])             # wybrane statystyki opisowe

min      24
max      55
count    10
Name: wiek, dtype: int64

---


## Więcej statystyk opisowych

Więcej statystyk opisowych, np. współczynnik zmienności i rozstęp międzykwartylowy, można znaleźć w pakiecie scipy.stats: \
https://docs.scipy.org/doc/scipy/reference/stats.html#summary-statistics

---

## Statystyki opisowe na DataFrame

Powyższe funkcje statystyczne z pandas istnieją również dla pandas.dataframe:

In [9]:
dane.mean(numeric_only = True)

wiek    36.2
staż    23.1
dtype: float64

In [10]:
dane[['wiek', 'staż']].mean()                 

wiek    36.2
staż    23.1
dtype: float64

In [11]:
dane.loc[dane['staż'] > 4, ['wiek', 'staż']].mean()    

wiek    38.125
staż    28.500
dtype: float64

---

# Funkcja groupby

Funkcja **groupby** grupuje wiersze na podstawie tych samych wartości w podanej kolumnie lub kolumnach. Po niej powinna nastąpić agregacja, która wtedy zostanie wykonana osobno dla każdej grupy:

In [12]:
dane.groupby('zawód')['wiek'].mean()               

zawód
elektryk       37.250000
programista    35.666667
sprzedawca     35.333333
Name: wiek, dtype: float64

### Kilka zmiennych grupujących

In [13]:
dane.groupby(['zawód', 'płeć'])['wiek'].mean()   

zawód        płeć
elektryk     K       29.000000
             M       40.000000
programista  K       35.666667
sprzedawca   K       55.000000
             M       25.500000
Name: wiek, dtype: float64

### Kilka zmiennych agregowanych

In [14]:
dane.groupby('zawód')[['wiek', 'staż']].mean()              

,wiek,staż
zawód,,
elektryk,37.250000,26.750000
programista,35.666667,22.000000
sprzedawca,35.333333,19.333333


### Kilka funkcji agregujących

In [15]:
dane.groupby('zawód')['wiek'].aggregate(['min', 'max', 'count'])     

,min,max,count
zawód,,,
elektryk,29,51,4
programista,25,49,3
sprzedawca,24,55,3


### Kilka zmiennych grupujących, kilka zmiennych agregujących, kilka funkcji agregujących

In [19]:
dane.groupby(['zawód', 'płeć'])[['wiek', 'staż']].aggregate(['min', 'max', 'count']).reset_index() 

zawód płeć wiek           staż          
                     min max count  min max count
0     elektryk    K   29  29     1   29  29     1
1     elektryk    M   33  51     3   23  32     3
2  programista    K   25  49     3    2  52     3
3   sprzedawca    K   55  55     1   48  48     1
4   sprzedawca    M   24  27     2    1   9     2

---

## Ćwiczenie

1. Znajdź waria stażu i wieku dla osób poszczególnych płci.

2. Znajdź liczność, medianę, wariancję oraz skośność lat stażu dla osób poszczególnych płci i zawodów.

---

# Funkcja apply

Funkcja **apply** wykonuje podaną funkcję w obrębie kolumny - pozwala na połączenie właściwości funkcji map z funkcją agregującą.

In [ ]:
dane[['wiek', 'staż']].map(lambda x: x + 1)                  # działania tylko na każdym elemencie osobno

In [ ]:
dane[['wiek', 'staż']].apply(lambda x: x + sum(x))           # działania na każdym elemencie z funkcją agregacyjną wywołaną w obrębie kolumny

Funkcję apply można też połączyć z funkcją groupby, aby funkcje agregacyjne zostały wywołane w obrębie otrzymanych przez nią podzbiorów.

In [ ]:
dane.groupby('zawód')['staż'].apply(lambda x: x + sum(x))

# Funkcja transform

Jeśli chcemy wynik funkcji apply z funkcją groupby dopisać jako kolumnę do danych potrzebujemy w zamian niej funkcji transform:

In [ ]:
dane['staż z sumą'] = dane.groupby('zawód')['staż'].transform(lambda x: x + sum(x))
dane['staż z sumą'] 

In [ ]:
dane

---

## Ćwiczenie

Wykonaj standaryzację wieku - od każdego wieku odejmij średnią i podziel otrzymany wynik przez odchylenie standardowe. 